# Experiment

### Prerequisites

**Imports**

In [ ]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, MultiLineString
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

**Fix directories, define path variables**

In [ ]:
# make sure notebook is ran from src
cwd = os.getcwd()
if not cwd.split(os.sep)[-1] == 'src':
    try:
        os.chdir('src')
    except FileNotFoundError:
        print('Error: please run from src dir or project root')
        sys.exit(1)

# define paths
path = lambda x: os.path.join(*x.split('/')) + os.sep
shp_path = path('../data/shapefiles')
plot_path = path('../plots')

**Read data**

In [ ]:
# these are just manhattan gdfs, with correct projection
STREETS: gpd.GeoDataFrame = gpd.read_file(shp_path + 'streets' + os.sep + 'M_streets.shp')
TRAFFIC: gpd.GeoDataFrame = gpd.read_file(shp_path + 'traffic' + os.sep + 'M_traffic.shp')
TREES: gpd.GeoDataFrame = gpd.read_file(shp_path + 'trees' + os.sep + 'M_trees.shp')

**Set Random Seed**

In [ ]:
seed = 2112
np.random.seed(seed)

---

### Grid Setup

**Create grid**

In [ ]:
# define a bounding box around Manhattan
xmin, ymin, xmax, ymax = STREETS.total_bounds
bbox = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)])

# define granularity of grid (units are degrees [EPSG:4326])
l, w = .005, .005

# create a grid of polygons
cols = list(np.arange(xmin, xmax+w, w))
rows = list(np.arange(ymin, ymax+l, l))
polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygons.append(Polygon([(x, y), (x, y+l), (x+w, y+l), (x+w, y), (x, y)]))

# create a gdf from the grid
GRID = gpd.GeoDataFrame({'geometry': polygons})
GRID.crs = 4326

# remove grid cells that do not intersect with any street
size_before = GRID.shape[0]
GRID = GRID[GRID.intersects(STREETS.unary_union)]
GRID = GRID.reset_index(drop=True)
print(f'Grid cells trimmed to {GRID.shape[0]} from {size_before}')

**Assign "easy" values to grid**

In [ ]:
# we would get a SettingWithCopyWarning, but it's not a problem
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # number of trees within grid cell
    GRID['n_trees'] = GRID.geometry.apply(lambda x: TREES[TREES.geometry.within(x)].shape[0])
    # average of all traffic volume probes within grid cell, multiplied by 96 (15 min intervals in a day)
    GRID['trfc_raw'] = GRID.geometry.apply(lambda x: TRAFFIC[TRAFFIC.geometry.within(x)].Avg_Vol.mean() * 96)

**Plot to check**

In [ ]:
# this dict will be used later as well, so we define all titles here
titles: dict[str] = {
    'n_trees': 'Number of Trees',
    'trfc_raw': 'Traffic Volume (raw)',
    'tree_dist': 'Average Distance between Trees',
    'tree_distl': 'Average Distance between Trees (log)',
    'street_len': 'Street Length per Grid Cell',
    'trfc_itp': 'Traffic Volume (interpolated)',
}

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
for ax, col, cmap in zip([ax1, ax2], ['n_trees', 'trfc_raw'], ['Greens', 'Reds']):
    GRID.plot(ax=ax, column=col, cmap=cmap, legend=True, zorder=1, legend_kwds={'orientation': 'horizontal', 'shrink': 0.5})
    STREETS.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
    ax.set_axis_off()
    ax.set_title(titles[col])
fig.tight_layout()
fig.savefig(plot_path + 'grid_raw.png', dpi=500)

---

### Street lengths per grid cell

**Define `haversine` function**

In [ ]:
def haversine(p1: tuple[float], p2: tuple[float]) -> float:
    """ Calculate the great circle distance in meters between two points on the earth. """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [p1[0], p1[1], p2[0], p2[1]])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371  # radius of earth in kilometers
    return c * r * 1000

**Calculate total street length per cell**

A more succinct definition will follow later.
This is just a proof of concept with four random cells to visualize how we get to the street lengths.

In [ ]:
# take three random grid cells
cells = [row.geometry for _, row in GRID.sample(3).iterrows()]

street_name_replacements = {
    'E ': 'East ', 'W ': 'West ', 'N ': 'North ', 'S ': 'South ',
    'St': 'Street', 'Ave': 'Avenue', 'Pl': 'Place', 'Rd': 'Road',
    'Dr': 'Drive', 'Blvd': 'Boulevard', 'Ct': 'Court', 'Ln': 'Lane',
    'Pkwy': 'Parkway', 'Ter': 'Terrace', 'Cir': 'Circle', 'Plz': 'Plaza',
    'Hwy': 'Highway', 'Expy': 'Expressway', 'Brg': 'Bridge', 'Trl': 'Trail',
    'Vyd': 'Viaduct', 'Mnr': 'Manor', 'Tunl': 'Tunnel', 'Rte': 'Route'
}

def plot_cell_streets(ax: plt.Axes, cell_geometry: Polygon, label: str) -> None:
    """ Plot the streets of a grid cell. """

    # create patch object of boundary of cell (cell.exterior.coords)
    ax.add_patch(plt.Polygon(cell_geometry.exterior.coords, facecolor='none', edgecolor='black', linewidth=1, zorder=2))
    ax.set_axis_off()

    # plot the cell's street segments
    streets = STREETS[STREETS.geometry.intersects(cell_geometry)]
    streets.plot(ax=ax, color='grey', alpha=0.5, zorder=1)
    
    segments = [cell_geometry.intersection(line) for line in streets.geometry]
    distances, centroids = [], []

    colors = [f'C{i}' for i in range(12)]
    ci = 0

    def recurse(segment):
        """ Recursively parse the segments of a MultiLineString. """
        nonlocal ci
        if isinstance(segment, LineString):
            start, end = segment.coords[0], segment.coords[-1]
            distances.append(haversine(start, end))
            centroids.append(((start[0] + end[0]) / 2, (start[1] + end[1]) / 2))
            ax.plot(*zip(*segment.coords), color=colors[ci%12], zorder=3)
            ci += 1
        elif isinstance(segment, MultiLineString):
            # MultiLineString has no coords, but multiple LineStrings which do
            for seg_part in segment.geoms:
                recurse(seg_part)
        else:
            raise ValueError(f'Unexpected type: {type(segment)}, expected LineString or MultiLineString')

    for segment in segments:
        recurse(segment)

    ci = 0
    # show length of each street segment at its midpoint
    for distance, centroid in zip(distances, centroids):
        # distance with transparent colored patch to make text more readable
        ax.text(centroid[0], centroid[1], f'{distance:.0f}', fontsize=8, color='black', ha='center', va='center', zorder=4, 
                bbox=dict(facecolor=colors[ci%12], edgecolor='none', alpha=0.2, boxstyle='round, pad=0.2'))
        ci += 1

    # find the street name that occurs most often in the segments
    street_names = streets['full_stree'].value_counts().sort_values(ascending=False).index
    street_name: str = street_names[0].lower().title()
    for abbr, full in street_name_replacements.items():
        street_name = street_name.replace(abbr, full)
    ax.set_title(
        f'{street_name} ({label})\ntotal length: {sum(distances):.0f} $m$',
        fontsize=10, color='black', ha='center', va='center', zorder=5
    )

    return

# create figure object
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
for ax, cell_geometry, label in zip(axes.flatten(), cells, ['A', 'B', 'C']):
    plot_cell_streets(ax, cell_geometry, label)
# plot grid in gray on fourth subplot
GRID.plot(ax=axes[1, 1], color='grey', alpha=0.1, zorder=1)
# plot the three sampled cells in blue
for cell_geometry, label in zip(cells, ['A', 'B', 'C']):
    axes[1, 1].add_patch(plt.Polygon(cell_geometry.exterior.coords, facecolor='blue', edgecolor='blue', linewidth=1, zorder=2))
    x, y = cell_geometry.centroid.coords[0]
    axes[1, 1].text(x, y, label, fontsize=10, color='white', ha='center', va='center', zorder=3)
axes[1, 1].set_title(f'Sampled Cells (seed {seed})', fontsize=10, color='black', ha='center', va='center', zorder=5)
axes[1, 1].set_axis_off()
fig.tight_layout()
fig.subplots_adjust(wspace=0.05, hspace=0.05)
fig.suptitle('Length Definition', fontsize=16, color='black', weight='bold', ha='center', va='center')
fig.savefig(plot_path + 'length_definition.pdf', dpi=500)

Here we see that the clipping does indeed work as intended.
There is also no overlap between the segments.
One shortcoming however is that the segments are calculated by taking the haversine distance between the start and end points of each segment, which is not robust against curvature in the roads.
Manhattan's famously grid-like nature minimizes this effect, but it is important to keep in mind.

**Apply to `GRID`**

In [ ]:
def get_total_street_length(cell_geometry: Polygon):
    """ Calculate the total length of all street segments within a cell. """

    streets = STREETS[STREETS.geometry.intersects(cell_geometry)]
    segments = [cell_geometry.intersection(line) for line in streets.geometry]
    total_length = 0

    def recurse(segment):
        """ Recursively parse the segments of a MultiLineString. """
        nonlocal total_length
        if isinstance(segment, LineString):
            start, end = segment.coords[0], segment.coords[-1]
            total_length += haversine(start, end)
        elif isinstance(segment, MultiLineString):
            # MultiLineString has no coords, but multiple LineStrings which do
            for seg_part in segment.geoms:
                recurse(seg_part)
        else:
            raise ValueError(f'Unexpected type: {type(segment)}, expected LineString or MultiLineString')
    
    for segment in segments:
        recurse(segment)

    return total_length

GRID['street_len'] = GRID.geometry.apply(get_total_street_length)

**Inspect results**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

GRID.plot(ax=ax1, column='street_len', cmap='Purples', legend=True, zorder=1, legend_kwds={'orientation': 'horizontal', 'shrink': 0.5})
STREETS.plot(ax=ax1, color='grey', alpha=0.2, linewidth=0.5, zorder=2)
ax1.set_axis_off()

bins = GRID.street_len.hist(ax=ax2, bins=50, color='grey')
for i, patch in enumerate(bins.patches):
    patch.set_facecolor(plt.cm.Purples(i / len(bins.patches)))
    patch.set_edgecolor('black')
ax2.set_axisbelow(True)
ax2.set_xlabel('Street Length (m)')
ax2.set_ylabel('Count')

# fig.set_size_inches(10, 5)
fig.subplots_adjust(wspace=0.05, hspace=0.05)

fig.suptitle('Total Street Length per Grid Cell', fontsize=16, color='black', weight='bold', ha='center', va='center', y=0.95)
fig.tight_layout()
fig.savefig(plot_path + 'total_street_length.pdf', dpi=500)

---

### Tree Distances

**Cell dimensions**

Now that we have the `haversine` function, we can also define the dimension of our grid cells.

In [ ]:
cell = GRID.iloc[0].geometry
cell_length = haversine(cell.exterior.coords[0], cell.exterior.coords[1])
cell_width = haversine(cell.exterior.coords[0], cell.exterior.coords[3])
cell_area = cell_length * cell_width
print(f'Cell length: {cell_length:.0f} m, cell width: {cell_width:.0f} m, cell area: {cell_area:.0f} m²')

**Tree diameter**

In the case study of Özdemir ([2019](https://www.sciencedirect.com/science/article/pii/S0048969718351350)), the distance between trees ("gap") was calculated with the diameter of the trees in mind.
Because we have no information about the diameter of the trees in our data, we will assume that the trees are of equal diameter.
This average diameter can be calculated by taking the total canopy area of Manhattan and dividing it by the total number of trees.
Data about the canopy area of Manhattan was taken from _The Nature Conservancy_: [The State of the Urban Forest in New York](https://www.nature.org/content/dam/tnc/nature/en/photos/TheStateoftheNYCUrbanForest.pdf).

In [ ]:
land_area: int = 3820  # acres
land_area *= 4046.86  # m²
total_canopy = land_area * .2018
# divide by total number of trees
average_tree_canopy = total_canopy / TREES.shape[0]
# diameter, assuming a perfect circle
average_tree_diameter = 2 * np.sqrt(average_tree_canopy / np.pi)
print(f'Average tree diameter: {average_tree_diameter:.2f} m')

**Filter data**

Based on the results of the previous section, we can calculate the average distance between two roadside trees in a grid cell by dividing the length of the street segments by the number of trees.
A slight problem is that some cells have little to zero trees.
This would lead to a division by zero error, or very large numbers that would throw off the scale of our model.

To avoid this, we can set a minimum number of trees per cell, in our case 10.
We can justify this by assuming that, if there are less than 10 roadside trees in an area of $0.23 \ \mathrm{km}^2$ (the area of a grid cell), then the area is probably not suitable for planting trees in the first place.

In [ ]:
GRID = GRID[GRID.n_trees >= 10]
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    GRID['tree_dist'] = GRID.apply(lambda row: max(row['street_len'] / row['n_trees'] - average_tree_diameter, 0.1), axis=1)

**Inspect results**

First we want to do a simple histogram of the calculated average distances.
Because we expect the distribution to still be heavily left-skewed, we will calculate the logarithm of the distances and plot the distribution of these as well.

In [ ]:
# create column with log of tree distance
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    GRID['tree_distl'] = np.log(GRID.tree_dist)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
GRID.tree_dist.hist(ax=ax1, bins=50, color='grey')
ax1.axvline(mean:=GRID.tree_dist.mean(), color='black', linestyle='dotted', linewidth=1, label=f'mean: {mean:.0f} m')
ax1.axvline(median:=GRID.tree_dist.median(), color='black', linestyle='dashed', linewidth=1, label=f'median: {median:.0f} m')
ax1.set_xlabel('Distance $[m]$')
ax1.set_ylabel('Count')
ax1.legend()
ax1.set_title('Linear Scale')

GRID.tree_distl.hist(ax=ax2, bins=50, color='grey')
ax2.set_xlabel('Distance $[\log(m)]$')
ax2.set_ylabel('Count')
ax2.set_title('Log Scale')

fig.suptitle('Distance between Trees')
fig.tight_layout()

Secondly, it should be very interesting to see the data plotted on the map.
We also show the attributes that are used to calculate these values.
Here, we also use the logarithm of the distances for a more clear visualization.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 10))
for ax, col, cmap in zip(axes, ['n_trees', 'street_len', 'tree_distl'], ['Greens', 'Reds', 'Blues']):
    GRID.plot(ax=ax, column=col, cmap=cmap, legend=True, legend_kwds={'orientation': 'horizontal', 'shrink': 0.5}, zorder=1)
    STREETS.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
    ax.set_axis_off()
    ax.set_title(titles[col])
fig.tight_layout()
fig.subplots_adjust(wspace=0.05, hspace=0.05)
fig.savefig(plot_path + 'tree_distance1.pdf', dpi=500)

For completeness, we also create a plot similar to the total street length plot (histogram and map), but this time for the average distances between trees, both in the original scale and in the logarithmic scale.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10), height_ratios=[2, 1])
for ax, col in zip([axes[0][0], axes[0][1]], ['tree_dist', 'tree_distl']):
    GRID.plot(ax=ax, column=col, cmap='Blues', legend=True, legend_kwds={'orientation': 'vertical', 'shrink': 0.5}, zorder=1)
    STREETS.plot(ax=ax, color='grey', alpha=0.2, linewidth=0.5, zorder=2)
    ax.set_axis_off()
    ax.set_title(titles[col])
for ax, col, xlabel in zip([axes[1][0], axes[1][1]], ['tree_dist', 'tree_distl'], ['Distance $[m]$', 'Distance $[\log(m)]$']):
    bins = GRID[col].hist(ax=ax, bins=50, color='grey', zorder=2)
    for i, patch in enumerate(bins.patches):
        patch.set_facecolor(plt.cm.Blues(i / len(bins.patches)))
        patch.set_edgecolor('black')
    if col == 'tree_dist':
        ax.axvline(mean:=GRID[col].mean(), color='C1', linestyle='dotted', linewidth=2, label=f'mean: {mean:.0f} m', zorder=1)
        ax.axvline(median:=GRID[col].median(), color='C1', linestyle='dashed', linewidth=2, label=f'median: {median:.0f} m', zorder=1)
        ax.legend()
    ax.set_axisbelow(True)
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Count')
fig.tight_layout()
fig.subplots_adjust(wspace=0.15, hspace=0)
fig.savefig(plot_path + 'tree_distance2.pdf', dpi=500)

---

### Traffic Volume Interpolation

**Define Data Points**

In [ ]:
x, y = TRAFFIC.geometry.x.values, TRAFFIC.geometry.y.values
phi = TRAFFIC.Avg_Vol.values

assert x.shape[0] == y.shape[0] == phi.shape[0]

fig, ax = plt.subplots(figsize=(4, 5))
STREETS.plot(ax=ax, color='gray', linewidth=0.5, zorder=1)
cax = ax.scatter(x, y, c=phi, cmap='viridis', s=5, alpha=0.75, zorder=2)
ax.set_title('Observations')
ax.set_axis_off()
fig.colorbar(cax, ax=ax, shrink=0.4)
fig.tight_layout()

**Interpolate with Ordinary Kriging**

In [ ]:
OK = OrdinaryKriging(x, y, phi, variogram_model='exponential', verbose=False, enable_plotting=False)
minx, miny, maxx, maxy = GRID.total_bounds
X = np.arange(minx, maxx, 0.005)
Y = np.arange(miny, maxy-0.005, 0.005)
zstar, ss = OK.execute('grid', X, Y)

**Inspect results**

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10))
cax = ax.imshow(
    zstar,
    extent = (minx, maxx, miny, maxy),
    cmap = 'viridis',
    origin = 'lower',
    alpha = 0.9,
    zorder = 1
)
STREETS.plot(ax=ax, color='k', alpha=0.1, zorder=2)
for i, row in GRID.iterrows():
    x_, y_ = row.geometry.exterior.xy
    ax.plot(x_, y_, color='w', alpha=0.2, zorder=3)
ax.scatter(x, y, c=phi, cmap='viridis', edgecolors='k', s=10, zorder=3)
ax.set_axis_off()
ax.set_title('Traffic Volume Interpolation\n(Ordinary Kriging)')
fig.colorbar(cax, ax=ax, shrink=0.4)
fig.tight_layout()
fig.savefig(plot_path + 'ordinary_kriging.pdf', dpi=500)

The interpolated grid only covers a portion of the study area due to the selective trimming of the original data where we discarded grid cells with less than 10 trees.
In order to illustrate what traffic data we are left with, we also trim the interpolated traffic volume grid to the same area and plot it next to the averages we previously calculated.

**Add to `GRID`**

In [ ]:
GRID['trfc_itp'] = np.nan
for i, row in GRID.iterrows():
    x_, y_ = row.geometry.exterior.xy
    x_idx = np.argmin(np.abs(X - x_[0]))
    y_idx = np.argmin(np.abs(Y - y_[0]))
    # multiply by 96 to get the average daily traffic volume, because after kriging, the unit is 15 minutes
    GRID.loc[i, 'trfc_itp'] = zstar[y_idx, x_idx] * 96

**Visualize**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 8))
for ax, col, cmap in zip(axes, ['trfc_raw', 'trfc_itp'], ['Reds', 'Reds']):
    GRID.plot(ax=ax, column=col, cmap=cmap, legend=True, legend_kwds={'orientation': 'horizontal', 'shrink': 0.5}, zorder=1)
    STREETS.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
    ax.set_axis_off()
    ax.set_title(titles[col])
fig.tight_layout()
fig.subplots_adjust(wspace=0.05, hspace=0.05)
fig.savefig(plot_path + 'traffic_interpolation.pdf', dpi=500)

**Save `GRID`**

In [ ]:
target_path = shp_path + 'grid' + os.sep + 'M_grid.shp'
with warnings.catch_warnings():
    # some column names will be truncated, but it's no big deal
    warnings.simplefilter('ignore')
    GRID.to_file(target_path)

---

### $\mathrm{PM}_{2.5}$ Model

**Create mini dataframe**

We will use the results of the case study by Özdemir ([2019](https://www.sciencedirect.com/science/article/pii/S0048969718351350)) as a baseline for our model.
The study was conducted in the city of Istanbul, Turkey, and the results are summarized in the following table.

In [ ]:
df = pd.DataFrame(columns=['vehicles', 'pm_conc', 'tree_dist'], index=['C1', 'C2', 'C3'])
df.loc['C1'] = [37_653, 35.54, np.nan]
df.loc['C2'] = [44_399, 36.39, 0.8]
df.loc['C3'] = [49_877, 28.81, 0.01]
df.pm_conc = df.pm_conc.astype(float)
df.vehicles = df.vehicles.astype(int)
df.tree_dist = df.tree_dist.astype(float)
df['pm_conc_per_veh'] = df.pm_conc / df.vehicles
df['pm_conc_relief_per_veh'] = abs(df.pm_conc_per_veh - df.pm_conc_per_veh['C1'])
df.pm_conc_relief_per_veh = df.pm_conc_relief_per_veh.astype(float)
df

**Fit exponential decay function**

In [ ]:
td = df.tree_dist.values[1:]
cr = df.pm_conc_relief_per_veh.values[1:]
tdl = np.log(td)
crl = np.log(cr)
M, B = np.polyfit(tdl, crl, 1)

def pm_concentration_relief(tree_distance: float | np.ndarray, n_vehicles: float = 1) -> float | np.ndarray:
    """ Based on exponential model fitted to data from C2 and C3 """
    return np.exp(M * np.log(tree_distance) + B) * n_vehicles

**Plot**

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
x_ = np.linspace(0.01, 10, 100)
y_ = pm_concentration_relief(x_)
ax.scatter(df.tree_dist, df.pm_conc_relief_per_veh, color='k', s=20)
ax.text(df.tree_dist['C2'], df.pm_conc_relief_per_veh['C2'], 'C2', ha='left', va='bottom', fontsize=12)
ax.text(df.tree_dist['C3'], df.pm_conc_relief_per_veh['C3'], 'C3', ha='left', va='top', fontsize=12)
ax.plot(x_, y_, color='k', linestyle='--', label=f'$y =$ {np.exp(B):.2e}$\cdot x^{{{M:.2f}}}$')
ax.set_xlim(-.1, 10.1)
ax.set_xlabel('Tree distance $[m]$')
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax.yaxis.major.formatter._useMathText = True
ax.set_ylabel('PM$_{2.5}$ concentration relief $[\mu g/m^3]$')
ax.legend()
ax.set_title('PM$_{2.5}$ concentration relief per vehicle\nas a function of tree distance');
fig.tight_layout()
fig.savefig(plot_path + 'pm_conc_relief.pdf', dpi=500)

**Define expected impact function**

In [ ]:
def expected_impact(row: pd.Series) -> float:
    """ Calculate expected impact of adding a tree to a grid cell """
    current_relief = pm_concentration_relief(row.tree_dist, row.trfc_itp)
    # calculate new tree distance, street length is same, trees += 1
    new_tree_dist = row.street_len / (row.n_trees + 1) - average_tree_diameter
    new_relief = pm_concentration_relief(new_tree_dist, row.trfc_itp)
    return new_relief - current_relief

**Visualize impact of one tree**

In [ ]:
GRID_ = GRID.copy()
with warnings.catch_warnings():
    # divide by zero warning, but it's no big deal
    warnings.simplefilter('ignore')
    GRID_['expected_impact'] = GRID_.apply(expected_impact, axis=1)
fig, ax = plt.subplots(figsize=(6, 8))
GRID_.plot(ax=ax, column='expected_impact', cmap='YlGn', legend=True, zorder=1,
           legend_kwds={'orientation': 'vertical', 'shrink': 0.5, 'label': r'PM$_{2.5}$ relief $[\mu g/m^3/\mathrm{day}]$'})
STREETS.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
ax.set_axis_off()
fig.suptitle('Expected impact of planting one tree')
fig.tight_layout()
fig.savefig(plot_path + 'expected_impact1.pdf', dpi=500)

**Generalize to $\mathbf{n}$ trees**

In [ ]:
def recommender_system(grid: gpd.GeoDataFrame, budget: int) -> tuple[gpd.GeoDataFrame, float]:
    """
    Iteratively, plant trees in grid cells with highest expected impact.
    Return the updated grid and the total impact achieved.
    """
    grid_ = grid.copy()
    grid_['planted'] = 0
    total_impact = 0
    for _ in range(budget):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            grid_['expected_impact'] = grid_.apply(expected_impact, axis=1)
        recommended_cell = grid_.expected_impact.idxmax()
        grid_.loc[recommended_cell, 'n_trees'] += 1
        grid_.loc[recommended_cell, 'planted'] += 1
        grid_['tree_dist'] = grid_.street_len / grid_.n_trees - average_tree_diameter
        total_impact += grid_.loc[recommended_cell, 'expected_impact']
    return grid_, total_impact

In [ ]:
budget = 10000
GRID_, total_impact = recommender_system(GRID, budget)
fig, ax = plt.subplots(figsize=(6, 8))
GRID_.plot(ax=ax, column='planted', cmap='YlGn', legend=True, zorder=1,
           legend_kwds={'orientation': 'vertical', 'shrink': 0.5, 'label': 'Number of trees'})
STREETS.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
ax.set_axis_off()
fig.suptitle(f'Recommended planting sites\nfor a budget of {budget} trees\n(Total impact: {total_impact:.2f} ' + r'$\mu g/m^3/\mathrm{day}$)')
fig.tight_layout()
fig.savefig(plot_path + 'expected_impact2.pdf', dpi=500)

**Benchmark**

In [ ]:
def random_recommender_system(grid: gpd.GeoDataFrame, budget: int) -> tuple[gpd.GeoDataFrame, float]:
    """
    Randomly, plant trees in grid cells.
    Return the updated grid and the total impact achieved.
    """
    grid_ = grid.copy()
    grid_['planted'] = 0
    total_impact = 0
    for _ in range(budget):
        random_cell = np.random.choice(grid_.index)
        grid_.loc[random_cell, 'n_trees'] += 1
        grid_.loc[random_cell, 'planted'] += 1
        grid_['tree_dist'] = grid_.street_len / grid_.n_trees - average_tree_diameter
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            impact_cell = expected_impact(grid_.loc[random_cell])
        if not np.isnan(impact_cell): total_impact += impact_cell
    return grid_, total_impact

In [ ]:
budget = 10000
GRID_, total_impact = random_recommender_system(GRID, budget)
fig, ax = plt.subplots(figsize=(6, 8))
GRID_.plot(ax=ax, column='planted', cmap='YlGn', legend=True, zorder=1,
           legend_kwds={'orientation': 'vertical', 'shrink': 0.5, 'label': 'Number of trees'})
STREETS.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
ax.set_axis_off()
fig.suptitle(f'Recommended planting sites\nfor a budget of {budget} trees\n(Total impact: {total_impact:.2f} ' + r'$\mu g/m^3/\mathrm{day}$)')
fig.tight_layout()
fig.savefig(plot_path + 'expected_impact3.pdf', dpi=500)

We see that our recommender system does indeed perform better than the baseline model, which is to be expected, as it simply always takes the best possible action by definition.
The recommended planting sites seem to share most correlation with the traffic volumes, which is to be expected as well.